In [1]:
import json
import os

SPLIT = "val/"

PUBTABNET = '../../pubtabnet/anns/'

ANN_PATH = PUBTABNET

json_list = os.listdir(ANN_PATH + SPLIT)

In [2]:
aux_list = []
html_list = []

for f in json_list:
    if(f[-6:-5] == "L"):
        html_list.append(f)
    else:
        aux_list.append(f)
json_list = aux_list

In [3]:
json_list[:10]

['PMC3752962_006_00.json',
 'PMC5285298_009_01.json',
 'PMC4721713_013_01.json',
 'PMC3189161_006_00.json',
 'PMC5921262_006_00.json',
 'PMC3940998_002_00.json',
 'PMC4211402_009_00.json',
 'PMC2796464_003_00.json',
 'PMC4714473_005_00.json',
 'PMC4900244_009_01.json']

In [4]:
from transformers import DonutProcessor

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [5]:
new_tokens  = ["<table_extraction>"]
new_tokens += ["<thead>", "</thead>", "<tbody>", "</tbody>"]
new_tokens += ["<tr>", "</tr>", "<td>", "</td>"]

new_tokens += ["<td ", ">"]
for i in range(10):
    new_tokens +=[' colspan="'+str(i)+'"']
    new_tokens +=[' rowspan="'+str(i)+'"']

new_tokens += ["<i>", "</i>", "<b>", "</b>", "<sup>", "</sup>", "<sub>", "</sub>"]
newly_added_num = processor.tokenizer.add_tokens(new_tokens)

In [6]:
print(new_tokens)

['<table_extraction>', '<thead>', '</thead>', '<tbody>', '</tbody>', '<tr>', '</tr>', '<td>', '</td>', '<td ', '>', ' colspan="0"', ' rowspan="0"', ' colspan="1"', ' rowspan="1"', ' colspan="2"', ' rowspan="2"', ' colspan="3"', ' rowspan="3"', ' colspan="4"', ' rowspan="4"', ' colspan="5"', ' rowspan="5"', ' colspan="6"', ' rowspan="6"', ' colspan="7"', ' rowspan="7"', ' colspan="8"', ' rowspan="8"', ' colspan="9"', ' rowspan="9"', '<i>', '</i>', '<b>', '</b>', '<sup>', '</sup>', '<sub>', '</sub>']


In [7]:
def write_msg(msg):
    with open("msg.json", 'w') as out:
        json.dump({'msg': msg}, out, ensure_ascii=False, indent=4)

In [8]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class DonutTableDataset(Dataset):
    def __init__(
        self,
        annotations,
        shuffle = True,
        split = "train",
        ignore_id = -100,
        prompt_end_token = None,
    ):            
        self.annotations = annotations
        
        self.split = split
        self.ignore_id = ignore_id
        
        
    def __len__(self):
        return len(self.annotations)
    
    
    def __getitem__(self, idx):
        
        file_name = self.annotations[idx]
        
        with open(ANN_PATH + SPLIT + file_name, encoding="utf-8") as f:
            annotation = json.load(f)
            
        target_sequence = annotation
        
        input_ids = processor.tokenizer(
            target_sequence,
            add_special_tokens=True,
            max_length= 15000,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )["input_ids"].squeeze(0)

        labels = input_ids.clone()
        labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id  
        
        encoding = dict(labels=labels,
                        file_name = file_name)
        
        return encoding

In [9]:
train_dataset = DonutTableDataset(html_list)

In [10]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [11]:
len_dics = []

def add_lens(file_name, tokens):
    for i in range(len(tokens)):
        j = 0
        for token in tokens[i]:
            j += 1
            if(token == -100):
                break

        len_dics.append({'file': file_name[i], 'len': j})

In [12]:
import torch
from tqdm.auto import tqdm


for i, batch in enumerate(tqdm(train_dataloader)):
    batch = {k: v for k, v in batch.items()}
    labels = batch["labels"]
    filename = batch["file_name"]
    add_lens(filename, labels)

  0%|          | 0/9115 [00:00<?, ?it/s]

In [13]:
import json
with open(ANN_PATH + SPLIT[:-1] + "-HTML_lens.json", 'w') as out:
    json.dump(len_dics, out, ensure_ascii=False, indent=4)

In [14]:
html_list[:10]

['PMC5738406_001_00-HTML.json',
 'PMC4665520_010_00-HTML.json',
 'PMC5098491_002_00-HTML.json',
 'PMC5602781_004_00-HTML.json',
 'PMC3117319_004_00-HTML.json',
 'PMC6020543_009_00-HTML.json',
 'PMC3913489_002_00-HTML.json',
 'PMC2745417_006_00-HTML.json',
 'PMC5452179_007_00-HTML.json',
 'PMC5551221_006_00-HTML.json']